# NN4: Loading and Analysing of NN Tuned with Random Search

In [3]:
# import libraries and modules needed for the project
import pandas as pd   # !pip install pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics

import tensorflow.keras.backend as K


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from keras_tuner import RandomSearch, Objective
import random
import os

from sklearn.metrics import precision_recall_curve

In [4]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# reading df again
df = pd.read_csv('diabetis_clean.csv')

# Selected Features
selected_features = [
    "severity_of_disease", "num_lab_procedures", "num_medications",
    "time_in_hospital", "number_inpatient", "visit_index", 
    "discharge_disposition_description",
    "number_diagnoses", "num_procedures", "age", "gender",
    "readmitted"
]

# create a copy of the dataframe with the selected features
df_selected = df[selected_features]

# Contineous Fearues that then need to be standardised
continuous_features = [
    "severity_of_disease", "num_lab_procedures", "num_medications",
    "time_in_hospital", "number_inpatient", "visit_index",
    "number_diagnoses", "num_procedures"
]

# Categorical Features need to be one hot encoded
categorical_features = [ "discharge_disposition_description", 
             "age", "gender" ]

# Split data into train and test sets
X = df_selected.drop('readmitted', axis=1)
y = df_selected['readmitted'].map({'<30': 1, 'NO': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Fit and transform the training data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

X_train = X_train.toarray()
X_test = X_test.toarray()

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


# Make sure type of data match with model expectations
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [5]:
# Define a function to build the model
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('input_units', min_value=32, max_value=512, step=32), activation='relu', input_dim=X_train.shape[1]))

    # Defining the number of hidden layers and their neurons
    for i in range(hp.Int('n_layers', 1, 5)):
        model.add(Dense(hp.Int(f'dense_{i}_units', min_value=32, max_value=512, step=32), activation='relu'))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=[Precision(name='precision'), Recall(name='recall'), tf.keras.metrics.AUC(curve='PR', name='f1')])
    
    return model


# Set up Keras Tuner
tuner = RandomSearch(
    build_model,
    objective=Objective("f1", direction="max"),
    max_trials=20,
    executions_per_trial=2,
    directory='output',
    project_name='HospitalReadmission')

Reloading Tuner from output/HospitalReadmission/tuner0.json


In [6]:
# Load the existing tuning results
tuner.reload()

In [7]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the model summary to see the configuration
best_model.summary()


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 384)            │        14,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 416)            │       160,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │        13,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,569 (740.50 KB)

 Trainable params: 189,569 (740.50 KB)

 Non-trainable params: 0 (0.00 B)